# 语音回复功能调试笔记本

本笔记本用于调试和测试语音回复功能的各项组件：
- 用户语音偏好设置 (VoicePreferenceService)
- 语音处理命令 (/voice_on, /voice_off, /voice)
- TTS服务 (Text-to-Speech)
- 语音回复辅助函数 (send_voice_or_text_reply)

## 功能说明
- 用户使用 `/voice_on` 开启语音回复，后续对话将使用语音进行回复
- 用户使用 `/voice_off` 关闭语音回复，后续对话将使用文本进行回复
- 语音偏好按 user_id + bot_username 存储，支持同一用户在不同Bot上有不同设置

In [ ]:
# 添加项目根目录到路径
import sys
import os
sys.path.insert(0, os.path.dirname(os.getcwd()))

# 设置必要的环境变量（用于测试）
os.environ.setdefault("TELEGRAM_BOT_TOKEN", "test_token")
os.environ.setdefault("OPENAI_API_KEY", "your_openai_key_here")

print("✅ 环境设置完成")

## 1. 测试语音偏好服务 (VoicePreferenceService)

In [ ]:
from src.services.voice_preference_service import VoicePreferenceService, voice_preference_service

print("✅ VoicePreferenceService 模块导入成功")
print(f"数据目录: {voice_preference_service.data_dir}")
print(f"偏好文件: {voice_preference_service.preferences_file}")

In [ ]:
# 创建一个临时的测试服务实例
test_service = VoicePreferenceService()

# 测试用户ID和Bot用户名
TEST_USER_ID = 123456789
TEST_BOT_USERNAME = "test_bot"

print(f"测试用户ID: {TEST_USER_ID}")
print(f"测试Bot用户名: {TEST_BOT_USERNAME}")

In [ ]:
# 测试默认状态（应该是关闭的）
default_status = test_service.is_voice_enabled(TEST_USER_ID, TEST_BOT_USERNAME)
print(f"默认语音状态: {default_status}")
assert default_status == False, "默认状态应该是关闭的"
print("✅ 默认状态测试通过")

In [ ]:
# 测试开启语音
test_service.set_voice_enabled(TEST_USER_ID, TEST_BOT_USERNAME, True)
enabled_status = test_service.is_voice_enabled(TEST_USER_ID, TEST_BOT_USERNAME)
print(f"开启后语音状态: {enabled_status}")
assert enabled_status == True, "开启后状态应该是True"
print("✅ 开启语音测试通过")

In [ ]:
# 测试关闭语音
test_service.set_voice_enabled(TEST_USER_ID, TEST_BOT_USERNAME, False)
disabled_status = test_service.is_voice_enabled(TEST_USER_ID, TEST_BOT_USERNAME)
print(f"关闭后语音状态: {disabled_status}")
assert disabled_status == False, "关闭后状态应该是False"
print("✅ 关闭语音测试通过")

In [ ]:
# 测试切换功能
new_state_1 = test_service.toggle_voice(TEST_USER_ID, TEST_BOT_USERNAME)
print(f"第一次切换后状态: {new_state_1}")
assert new_state_1 == True, "切换后应该是True"

new_state_2 = test_service.toggle_voice(TEST_USER_ID, TEST_BOT_USERNAME)
print(f"第二次切换后状态: {new_state_2}")
assert new_state_2 == False, "再次切换后应该是False"

print("✅ 切换功能测试通过")

In [ ]:
# 测试不同Bot的独立设置
BOT1 = "bot_one"
BOT2 = "bot_two"

test_service.set_voice_enabled(TEST_USER_ID, BOT1, True)
test_service.set_voice_enabled(TEST_USER_ID, BOT2, False)

status_bot1 = test_service.is_voice_enabled(TEST_USER_ID, BOT1)
status_bot2 = test_service.is_voice_enabled(TEST_USER_ID, BOT2)

print(f"Bot1 语音状态: {status_bot1}")
print(f"Bot2 语音状态: {status_bot2}")

assert status_bot1 == True, "Bot1应该开启语音"
assert status_bot2 == False, "Bot2应该关闭语音"
print("✅ 不同Bot独立设置测试通过")

## 2. 测试语音命令处理器

In [ ]:
from src.handlers.voice_handler import (
    voice_on_command,
    voice_off_command,
    voice_command,
    voice_callback,
    get_voice_handlers
)

print("✅ Voice Handler 模块导入成功")
print(f"可用处理器数量: {len(get_voice_handlers())}")

In [ ]:
from unittest.mock import AsyncMock, MagicMock, patch
import asyncio

async def test_voice_on_command():
    """
    测试 /voice_on 命令
    验证:
    1. 正确设置语音偏好为开启
    2. 发送正确的确认消息
    """
    # 模拟 Update 和 Context
    mock_update = MagicMock()
    mock_update.effective_user.id = 12345
    mock_update.message = AsyncMock()
    
    mock_context = MagicMock()
    mock_context.bot.username = "test_bot"
    
    with patch('src.handlers.voice_handler.voice_preference_service') as mock_pref:
        await voice_on_command(mock_update, mock_context)
        
        # 验证设置了语音偏好为开启
        mock_pref.set_voice_enabled.assert_called_once_with(12345, "test_bot", True)
        
        # 验证发送了文本确认消息
        mock_update.message.reply_text.assert_called_once()
        confirmation_text = mock_update.message.reply_text.call_args[0][0]
        
        assert "语音回复功能已开启" in confirmation_text, f"确认消息应包含'语音回复功能已开启', 实际: {confirmation_text}"
        assert "后续的对话将使用语音进行回复" in confirmation_text, f"确认消息应包含'后续的对话将使用语音进行回复', 实际: {confirmation_text}"
        
        print(f"✅ /voice_on 命令测试通过")
        print(f"   确认消息: {confirmation_text}")

await test_voice_on_command()

In [ ]:
async def test_voice_off_command():
    """
    测试 /voice_off 命令
    验证:
    1. 正确设置语音偏好为关闭
    2. 发送正确的确认消息
    """
    # 模拟 Update 和 Context
    mock_update = MagicMock()
    mock_update.effective_user.id = 12345
    mock_update.message = AsyncMock()
    
    mock_context = MagicMock()
    mock_context.bot.username = "test_bot"
    
    with patch('src.handlers.voice_handler.voice_preference_service') as mock_pref:
        await voice_off_command(mock_update, mock_context)
        
        # 验证设置了语音偏好为关闭
        mock_pref.set_voice_enabled.assert_called_once_with(12345, "test_bot", False)
        
        # 验证发送了文本确认消息
        mock_update.message.reply_text.assert_called_once()
        confirmation_text = mock_update.message.reply_text.call_args[0][0]
        
        assert "语音回复功能已关闭" in confirmation_text, f"确认消息应包含'语音回复功能已关闭', 实际: {confirmation_text}"
        assert "后续的对话将使用文本进行回复" in confirmation_text, f"确认消息应包含'后续的对话将使用文本进行回复', 实际: {confirmation_text}"
        
        print(f"✅ /voice_off 命令测试通过")
        print(f"   确认消息: {confirmation_text}")

await test_voice_off_command()

## 3. 测试语音回复辅助函数

In [ ]:
from src.utils.voice_helper import send_voice_or_text_reply

print("✅ Voice Helper 模块导入成功")

In [ ]:
async def test_voice_disabled_sends_text():
    """
    测试当语音关闭时，发送文本回复
    """
    mock_message = AsyncMock()
    mock_bot = MagicMock()
    mock_bot.bot_username = "test_bot"
    mock_bot.voice_id = "xiaoyan"
    
    test_response = "这是一个测试回复"
    test_user_id = 12345
    
    with patch('src.utils.voice_helper.voice_preference_service') as mock_pref:
        # 模拟语音关闭
        mock_pref.is_voice_enabled.return_value = False
        
        result = await send_voice_or_text_reply(
            message=mock_message,
            response=test_response,
            bot=mock_bot,
            user_id=test_user_id
        )
        
        # 验证检查了语音偏好
        mock_pref.is_voice_enabled.assert_called_once_with(test_user_id, "test_bot")
        
        # 验证发送了文本回复
        mock_message.reply_text.assert_called_once_with(test_response)
        
        # 验证返回类型
        assert result == "text", f"返回类型应该是'text', 实际: {result}"
        
        print("✅ 语音关闭时发送文本测试通过")

await test_voice_disabled_sends_text()

In [ ]:
async def test_voice_enabled_sends_voice():
    """
    测试当语音开启时，发送语音回复
    """
    mock_message = AsyncMock()
    mock_bot = MagicMock()
    mock_bot.bot_username = "test_bot"
    mock_bot.voice_id = "xiaoyan"
    
    test_response = "这是一个测试回复"
    test_user_id = 12345
    test_audio_data = b"fake_audio_data"
    
    with patch('src.utils.voice_helper.voice_preference_service') as mock_pref, \
         patch('src.utils.voice_helper.tts_service') as mock_tts:
        
        # 模拟语音开启
        mock_pref.is_voice_enabled.return_value = True
        
        # 模拟TTS服务
        mock_tts.generate_voice = AsyncMock(return_value=test_audio_data)
        mock_tts.get_voice_as_buffer.return_value = MagicMock()
        
        result = await send_voice_or_text_reply(
            message=mock_message,
            response=test_response,
            bot=mock_bot,
            user_id=test_user_id
        )
        
        # 验证检查了语音偏好
        mock_pref.is_voice_enabled.assert_called_once_with(test_user_id, "test_bot")
        
        # 验证调用了TTS服务
        mock_tts.generate_voice.assert_called_once()
        
        # 验证发送了语音回复
        mock_message.reply_voice.assert_called_once()
        
        # 验证返回类型
        assert result == "voice", f"返回类型应该是'voice', 实际: {result}"
        
        print("✅ 语音开启时发送语音测试通过")

await test_voice_enabled_sends_voice()

In [ ]:
async def test_voice_fallback_on_error():
    """
    测试当语音生成失败时，回退到文本回复
    """
    mock_message = AsyncMock()
    mock_bot = MagicMock()
    mock_bot.bot_username = "test_bot"
    mock_bot.voice_id = "xiaoyan"
    
    test_response = "这是一个测试回复"
    test_user_id = 12345
    
    with patch('src.utils.voice_helper.voice_preference_service') as mock_pref, \
         patch('src.utils.voice_helper.tts_service') as mock_tts:
        
        # 模拟语音开启
        mock_pref.is_voice_enabled.return_value = True
        
        # 模拟TTS服务失败
        mock_tts.generate_voice = AsyncMock(return_value=None)
        
        result = await send_voice_or_text_reply(
            message=mock_message,
            response=test_response,
            bot=mock_bot,
            user_id=test_user_id
        )
        
        # 验证回退到文本回复
        mock_message.reply_text.assert_called_once_with(test_response)
        
        # 验证返回类型
        assert result == "text", f"返回类型应该是'text', 实际: {result}"
        
        print("✅ TTS失败回退测试通过")

await test_voice_fallback_on_error()

## 4. 测试 TTS 服务

In [ ]:
from src.services.tts_service import TTSService, tts_service

print("✅ TTS Service 模块导入成功")
print(f"当前TTS提供商: {tts_service.provider}")
print(f"默认音色: {tts_service.default_voice}")
print(f"可用音色: {tts_service.get_available_voices()}")

In [ ]:
# 测试音色验证
valid_voices = ["xiaoyan", "xiaoyu", "alloy", "nova"]
invalid_voices = ["invalid_voice", "xyz"]

print("测试有效音色:")
for voice in valid_voices:
    is_valid = tts_service.is_voice_id_valid(voice)
    print(f"  {voice}: {'✅ 有效' if is_valid else '❌ 无效'}")

print("\n测试无效音色:")
for voice in invalid_voices:
    is_valid = tts_service.is_voice_id_valid(voice)
    print(f"  {voice}: {'✅ 有效' if is_valid else '❌ 无效'}")

## 5. 端到端流程测试

In [ ]:
async def test_full_voice_flow():
    """
    端到端测试完整的语音功能流程:
    1. 用户发送 /voice_on -> 语音开启
    2. 发送消息 -> 收到语音回复
    3. 用户发送 /voice_off -> 语音关闭
    4. 发送消息 -> 收到文本回复
    """
    print("=" * 50)
    print("端到端语音功能流程测试")
    print("=" * 50)
    
    test_user_id = 99999
    test_bot_username = "e2e_test_bot"
    
    # 使用全局服务实例
    from src.services.voice_preference_service import voice_preference_service as vps
    
    # 步骤1: 验证初始状态是关闭的
    initial_status = vps.is_voice_enabled(test_user_id, test_bot_username)
    print(f"\n1. 初始状态: 语音{'开启' if initial_status else '关闭'}")
    assert initial_status == False, "初始状态应该是关闭"
    print("   ✅ 初始状态正确")
    
    # 步骤2: 模拟 /voice_on 命令
    vps.set_voice_enabled(test_user_id, test_bot_username, True)
    after_on = vps.is_voice_enabled(test_user_id, test_bot_username)
    print(f"\n2. 执行 /voice_on 后: 语音{'开启' if after_on else '关闭'}")
    assert after_on == True, "执行/voice_on后应该开启"
    print("   ✅ /voice_on 执行成功")
    print("   预期消息: '🎤 语音回复功能已开启，后续的对话将使用语音进行回复'")
    
    # 步骤3: 测试语音回复逻辑
    mock_message = AsyncMock()
    mock_bot = MagicMock()
    mock_bot.bot_username = test_bot_username
    mock_bot.voice_id = "xiaoyan"
    
    with patch('src.utils.voice_helper.tts_service') as mock_tts:
        mock_tts.generate_voice = AsyncMock(return_value=b"audio_data")
        mock_tts.get_voice_as_buffer.return_value = MagicMock()
        
        result = await send_voice_or_text_reply(
            message=mock_message,
            response="测试消息",
            bot=mock_bot,
            user_id=test_user_id
        )
        
        print(f"\n3. 语音开启时发送消息: 回复类型={result}")
        assert result == "voice", "语音开启时应该返回voice"
        print("   ✅ 语音回复成功")
    
    # 步骤4: 模拟 /voice_off 命令
    vps.set_voice_enabled(test_user_id, test_bot_username, False)
    after_off = vps.is_voice_enabled(test_user_id, test_bot_username)
    print(f"\n4. 执行 /voice_off 后: 语音{'开启' if after_off else '关闭'}")
    assert after_off == False, "执行/voice_off后应该关闭"
    print("   ✅ /voice_off 执行成功")
    print("   预期消息: '📝 语音回复功能已关闭，后续的对话将使用文本进行回复'")
    
    # 步骤5: 测试文本回复逻辑
    mock_message_2 = AsyncMock()
    
    result_2 = await send_voice_or_text_reply(
        message=mock_message_2,
        response="测试消息2",
        bot=mock_bot,
        user_id=test_user_id
    )
    
    print(f"\n5. 语音关闭时发送消息: 回复类型={result_2}")
    assert result_2 == "text", "语音关闭时应该返回text"
    mock_message_2.reply_text.assert_called_once()
    print("   ✅ 文本回复成功")
    
    print("\n" + "=" * 50)
    print("✅ 端到端测试全部通过!")
    print("=" * 50)

await test_full_voice_flow()

## 6. 运行所有测试

In [ ]:
async def run_all_tests():
    """
    运行所有语音功能测试
    """
    print("\n" + "=" * 60)
    print("语音回复功能完整测试套件")
    print("=" * 60)
    
    tests = [
        ("语音偏好服务 - 默认状态", test_voice_disabled_sends_text),
        ("语音偏好服务 - 开启语音", test_voice_enabled_sends_voice),
        ("语音偏好服务 - 错误回退", test_voice_fallback_on_error),
        ("/voice_on 命令", test_voice_on_command),
        ("/voice_off 命令", test_voice_off_command),
        ("端到端流程", test_full_voice_flow),
    ]
    
    passed = 0
    failed = 0
    
    for name, test_func in tests:
        try:
            print(f"\n--- 测试: {name} ---")
            await test_func()
            passed += 1
        except Exception as e:
            failed += 1
            print(f"❌ 测试失败: {name}")
            print(f"   错误: {str(e)}")
    
    print("\n" + "=" * 60)
    print(f"测试结果: {passed}/{passed + failed} 通过")
    if failed == 0:
        print("🎉 所有测试通过!")
    else:
        print(f"⚠️ {failed} 个测试失败")
    print("=" * 60)

await run_all_tests()